In [155]:
import spidev
import pigpio
import os
from RpiMotorLib import RpiMotorLib
import json



TMC429_CS_PIN = 5   # GPIO 5
TMC429_CLK = 4      # GPIO 4
TMC2130_CS_PIN = 8  # GPIO 8
f_clk = 16000000


gpiod = pigpio.pi()


gpiod.set_mode(TMC429_CS_PIN, pigpio.OUTPUT)
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)

gpiod.set_mode(TMC429_CLK, pigpio.OUTPUT)
gpiod.hardware_clock(TMC429_CLK, f_clk)


gpiod.set_mode(TMC2130_CS_PIN, pigpio.OUTPUT)
gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)

def parse_status_byte(byte):
    status = {
        "INT": (byte & 0x80) != 0,
        "CDGW": (byte & 0x40) != 0,
        "RS3": (byte & 0x20) != 0,
        "RS2": (byte & 0x08) != 0,
        "RS1": (byte & 0x02) != 0,
        "xEQt3": (byte & 0x10) != 0,
        "xEQt2": (byte & 0x04) != 0,
        "xEQt1": (byte & 0x01) != 0
    }
    status = json.dumps(status, indent=2)
    print(status)

In [3]:
import spidev
import time

SPI_BUS = 0
SPI_DEVICE = 0
spi = spidev.SpiDev()
spi.open(SPI_BUS, SPI_DEVICE)
spi.max_speed_hz = 100000
spi.no_cs = True
spi.mode = 3





In [ ]:
# TMC 2130
register = 0x04

while True:
    # Sending the read command
    gpiod.write(TMC2130_CS_PIN, pigpio.LOW)
    # time.sleep(0.001)
    response = spi.xfer2([register, 0, 0, 0, 0])
    gpiod.write(TMC2130_CS_PIN, pigpio.HIGH)
    # print(response)
    print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])} {hex(response[4])}")
    time.sleep(1)

In [156]:
# TMC 429
# Read the TMC 429 version
register = 0b01110011

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")
print(parse_status_byte(response[0]))

0x14 0x42 0x91 0x1
{
  "INT": false,
  "CDGW": false,
  "RS3": false,
  "RS2": false,
  "RS1": false,
  "xEQt3": true,
  "xEQt2": true,
  "xEQt1": false
}
None


In [184]:
# TMC 429
# Read the TMC 429 Pulse_div and ramp_div
register = 0b00011001

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

pulse_div = (response[2] & 0xF0) >> 4
ramp_div = (response[2] & 0xF)
print(pulse_div)
print(ramp_div)

# Read the TMC 429 PMUL and PDIV
register = 0b000010011

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

pmul = (response[2])
pdiv = (response[3])
print(pmul)
print(pdiv)

0x14 0x0 0x0 0x0
0
0
0x14 0x0 0xfa 0x1
250
1


In [182]:
# TMC 429
# Set the TMC 429 Pulse_div and ramp_div
register = 0b00011000
byte3 = 0b00000000
byte2 = 0b00000000      
byte1 = 0b00000000 

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

# Set the TMC 429 PMUL and PDIV
register = 0b000010010
byte3 = 0b00000000
byte2 = 0b11111010
byte1 = 0b00000001

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0


In [185]:
# TMC 429
# Read the TMC 429 Target Speed
register = 0b00001001

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Tgt Speed: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Read the TMC 429 Min Speed
register = 0b00000101

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Min Speed: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Read the TMC 429 Max Speed
register = 0b00000111

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Max Speed: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Read the TMC 429 Max Accel
register = 0b00001101

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Max Accel: {hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

Tgt Speed: 0x14 0x0 0x0 0x0
Min Speed: 0x14 0x0 0x0 0x0
Max Speed: 0x14 0x0 0x7 0xff
Max Accel: 0x14 0x0 0x7 0xd0


In [207]:
# TMC 429
# Set the TMC 429 Target Speed

trgt_speed = 200

if trgt_speed > 0x7ff:
    trgt_speed = 0x7ff

register = 0b00001000
byte3 = 0b00000000
byte2 = trgt_speed >> 8
byte1 = trgt_speed & 0xFF


print(f"Max speed: {f_clk / (2**pulse_div * 32)}")

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"Tgt Speed: {float((f_clk*  trgt_speed) / ((2**pulse_div) * 2048 * 32)) }")
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")


# Set the TMC 429 Max Speed
register = 0b00000110
byte3 = 0b00000000
byte2 = 0x07
byte1 = 0xff

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

# Set the TMC 429 Min Speed
register = 0b00000100

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

# Set the TMC 429 Max Accel
register = 0b00001100
byte3 = 0b00000000
byte2 = 0x07
byte1 = 0xD0

# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{hex(response[0])} {hex(response[1])} {hex(response[2])} {hex(response[3])}")

Max speed: 500000.0
Tgt Speed: 48828.125
0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0
0x14 0x0 0x0 0x0


In [178]:
# TMC 429
# Set the TMC 429 to velocity mode
register = 0b00010100
byte3 = 0b00000000
byte2 = 0b00000000      # Disable endstop L and R
byte1 = 0b00000010      # Velocity mode

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)

register = 0b00010101
# Sending the read command
gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10100 0b0 0b0 0b10


In [119]:
# TMC 429
# Read the TMC 429 IF Config
register = 0b01101001

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, 0, 0, 0])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10100 0b0 0b0 0b100000


In [93]:
# TMC 429
# Set the TMC 429 in Step and Dir mode
register = 0b01101000
byte3 = 0b00000000
byte2 = 0b00000000
byte1 = 0b00100000

gpiod.write(TMC429_CS_PIN, pigpio.LOW)
response = spi.xfer2([register, byte3, byte2, byte1])
gpiod.write(TMC429_CS_PIN, pigpio.HIGH)
print(f"{bin(response[0])} {bin(response[1])} {bin(response[2])} {bin(response[3])}")

0b10101 0b0 0b0 0b0
